In [1]:
import os
from vedo import *
import vtk
from vmtk import pypes
import traceback
import pickle
import networkx as nx
from auxiliares import read_vtk, save_r_p, read_obj, vtpToObj
from parseObj import calcularMatriz
import Arbol as modelo

MESH OBJ to VTP

In [ ]:
files = os.listdir('mallasArregladas')
for file in files:
    if file.split(".")[1] == "obj" and (file.split(".")[0] + ".obj") not in files:
    #if file.split(".")[1] == "obj":
        print("convirtiendo")
        data = read_obj('mallasArregladas/'+file)
        file = file.split(".")[0]
        writer = vtk.vtkXMLPolyDataWriter();
        writer.SetFileName('mallasArregladas/'+file+".vtp");
        writer.SetInputData(data);
        writer.Write();

convirtiendo
convirtiendo
convirtiendo
convirtiendo
convirtiendo
convirtiendo
convirtiendo
convirtiendo
convirtiendo
convirtiendo
convirtiendo
convirtiendo
convirtiendo
convirtiendo
convirtiendo
convirtiendo
convirtiendo
convirtiendo
convirtiendo
convirtiendo
convirtiendo
convirtiendo
convirtiendo
convirtiendo
convirtiendo
convirtiendo
convirtiendo
convirtiendo
convirtiendo
convirtiendo
convirtiendo
convirtiendo
convirtiendo
convirtiendo
convirtiendo
convirtiendo
convirtiendo
convirtiendo
convirtiendo
convirtiendo
convirtiendo
convirtiendo
convirtiendo
convirtiendo
convirtiendo
convirtiendo
convirtiendo
convirtiendo
convirtiendo
convirtiendo
convirtiendo
convirtiendo
convirtiendo
convirtiendo
convirtiendo
convirtiendo
convirtiendo
convirtiendo
convirtiendo
convirtiendo
convirtiendo
convirtiendo
convirtiendo
convirtiendo
convirtiendo
convirtiendo
convirtiendo
convirtiendo
convirtiendo
convirtiendo
convirtiendo
convirtiendo
convirtiendo
convirtiendo
convirtiendo
convirtiendo
convirtiendo

Obtain centerline

In [ ]:
#files = ["ArteryObjAN1-11.obj"]
cfolder = os.listdir('crossSections')
for file in files:
    if file.split(".")[1] == "vtp":
        if file.split(".")[0] + '-network.vtp' not in cfolder:
            script = 'vmtknetworkextraction '
            input_file =  '-ifile mallasArregladas/' + file 
            output_file = ' -ofile centerlines/' + file.split(".")[0] + '-network.vtp'
            myPype = pypes.PypeRun(script+input_file+output_file)


Automatic piping vmtknetworkextraction
Parsing options vmtknetworkextraction
    SurfaceInputFileName = mallasArregladas/ArteryObjAN1-0.vtp
    NetworkOutputFileName = centerlines/ArteryObjAN1-0-network.vtp
Explicit piping vmtknetworkextraction
Input vmtknetworkextraction members:
    Id = 0
    Disabled = 0
    Surface = None
    SurfaceInputFileName = mallasArregladas/ArteryObjAN1-0.vtp
    AdvancementRatio = 1.05
    RadiusArrayName = Radius
    TopologyArrayName = Topology
    MarksArrayName = Marks
    NetworkOutputFileName = centerlines/ArteryObjAN1-0-network.vtp
    GraphLayoutOutputFileName = 
Reading VTK XML surface file.
Executing vmtknetworkextraction ...
Progress: 100%Progress: 100%
Done executing vmtknetworkextraction.
Writing VTK XML network file.
Output vmtknetworkextraction members:
    Id = 0
    Network = vtkPolyData
    GraphLayout = vtkPolyData

Automatic piping vmtknetworkextraction
Parsing options vmtknetworkextraction
    SurfaceInputFileName = mallasArregladas/

In [ ]:
for file in files:
    ##Load mesh and centerline
    msh = load("mallasArregladas/"+file)
    centerline = read_vtk("centerlines/" + file.split(".")[0] + "-network.vtp") 
    if file.split(".")[1] == "obj":
        if file.split(".")[0] + '-sections.obj' not in cfolder:
            try:
                save_r_p(centerline, msh, file)
                print("Calculated: ", file)
            except Exception:
                #traceback.print_exc()
                print("problem with: ", file)

problem with:  ArteryObjAN152-8.obj
problem with:  ArteryObjAN158-17.obj
problem with:  ArteryObjAN164-8.obj
problem with:  ArteryObjAN165-3.obj
problem with:  ArteryObjAN166-3.obj
problem with:  ArteryObjAN171-12.obj
problem with:  ArteryObjAN174-8.obj
problem with:  ArteryObjAN174-9.obj
problem with:  ArteryObjAN181-11.obj
problem with:  ArteryObjAN181-8.obj
problem with:  ArteryObjAN182-9.obj
problem with:  ArteryObjAN183-1.obj
Calculated:  ArteryObjAN186-16.obj
Calculated:  ArteryObjAN186-17.obj
Calculated:  ArteryObjAN187-1.obj
Calculated:  ArteryObjAN187-10.obj
Calculated:  ArteryObjAN187-13.obj
Calculated:  ArteryObjAN187-14.obj
Calculated:  ArteryObjAN187-15.obj
Calculated:  ArteryObjAN187-19.obj
Calculated:  ArteryObjAN187-2.obj
Calculated:  ArteryObjAN187-5.obj
Calculated:  ArteryObjAN187-6.obj
Calculated:  ArteryObjAN187-7.obj
Calculated:  ArteryObjAN187-9.obj
Calculated:  ArteryObjAN188-0.obj
Calculated:  ArteryObjAN188-10.obj
Calculated:  ArteryObjAN188-13.obj
Calculated: 

centerline to obj

In [ ]:
centerfolder = os.listdir("centerlines")
for file in centerfolder:
    if file.split(".")[1] == "vtp" and (file.split(".")[0] + '.obj') not in centerfolder:
        print("converting")
        vtpToObj(file)

converting
converting
converting
converting
converting
converting
converting
converting
converting
converting
converting
converting
converting
converting
converting
converting
converting
converting
converting
converting


create graph

In [ ]:
gfolder = os.listdir('grafos')
l_error = []
for file in files:
    if file.split(".")[1] is not None:
        if file.split(".")[1] == "obj":
            fileObj = open("centerlines/" +file.split(".")[0] +"-network.obj")
            if file.split(".")[0] + '-grafo.gpickle' not in gfolder:
            #if True:
                try: 
                    grafo = calcularMatriz(fileObj, "radius/" + file.split(".")[0] + "-radius.npy")
                    print("calculating: ", file)
                    with open("grafos/" + file.split(".")[0] + '-grafo.gpickle', 'wb') as f:
                        #print(grafo.nodes('radio'))
                        pickle.dump(grafo, f, pickle.HIGHEST_PROTOCOL)
                except Exception:
                    print("problem with: ", file)
                    l_error.append(file)
                    #traceback.print_exc()
gfolder = os.listdir('grafos')   
#print("problem with files: ", l_error)

problem with:  ArteryObjAN134-3.obj
problem with:  ArteryObjAN138-13.obj
problem with:  ArteryObjAN138-17.obj
problem with:  ArteryObjAN152-8.obj
problem with:  ArteryObjAN153-19.obj
problem with:  ArteryObjAN158-17.obj
problem with:  ArteryObjAN160-19.obj
problem with:  ArteryObjAN164-8.obj
problem with:  ArteryObjAN165-3.obj
problem with:  ArteryObjAN165-7.obj
problem with:  ArteryObjAN166-3.obj
problem with:  ArteryObjAN166-7.obj
problem with:  ArteryObjAN167-11.obj
problem with:  ArteryObjAN167-19.obj
problem with:  ArteryObjAN171-12.obj
problem with:  ArteryObjAN174-8.obj
problem with:  ArteryObjAN174-9.obj
problem with:  ArteryObjAN177-10.obj
problem with:  ArteryObjAN181-11.obj
problem with:  ArteryObjAN181-8.obj
problem with:  ArteryObjAN182-9.obj
problem with:  ArteryObjAN183-1.obj
calculating:  ArteryObjAN186-16.obj
calculating:  ArteryObjAN186-17.obj
calculating:  ArteryObjAN187-1.obj
calculating:  ArteryObjAN187-10.obj
calculating:  ArteryObjAN187-13.obj
calculating:  Arter

create binary tree

In [ ]:
t_list = os.listdir('Trees')

for file in files:
    if file.split(".")[0] + '-grafo.gpickle' in gfolder and file.split(".")[1] == 'obj' and (file.split(".")[0] + '_tree.dat') not in t_list:

        grafo = pickle.load(open('grafos/' + file.split(".")[0] + '-grafo.gpickle',  'rb'))
        grafo = grafo.to_undirected()

        nc = 0
        #controlo si tiene ciclos
        if len(nx.cycle_basis(grafo))>0:
            nc = 1
        

        nb = 0
        #controlo si es binario (no hay nodos con mas de 3 edges)
        for nodo in grafo.nodes:
            if len(grafo.edges(nodo))>3:
                nb = 1
                break
        
        if nb ==0 and nc == 0: 
            aRecorrer = []
            numeroNodoInicial = 1
            distancias = nx.floyd_warshall( grafo )

            parMaximo = (-1, -1)
            maxima = -1
        
            for nodoInicial in distancias.keys():
                for nodoFinal in distancias[nodoInicial]:
                    if distancias[nodoInicial][nodoFinal] > maxima:
                        maxima = distancias[nodoInicial] [nodoFinal]
                        parMaximo = (nodoInicial, nodoFinal)

            for nodo in grafo.nodes:
                if distancias[parMaximo[0]][nodo] == int( maxima / 2):
                    numeroNodoInicial = nodo
                    if len(grafo.edges(numeroNodoInicial))>2:
                        numeroNodoInicial = list(grafo.edges(numeroNodoInicial))[0][1]

                    break
        
    
            rad = list(grafo.nodes[numeroNodoInicial]['radio'])
            nodoRaiz = modelo.Node( numeroNodoInicial, radius =  rad )

            for vecino in grafo.neighbors( numeroNodoInicial ):
                if vecino != numeroNodoInicial:
                    aRecorrer.append( (vecino, numeroNodoInicial,nodoRaiz ) )
            
            while len(aRecorrer) != 0:
                nodoAAgregar, numeroNodoPadre,nodoPadre = aRecorrer.pop(0)
                radius = list(grafo.nodes[nodoAAgregar]['radio'])
   
                nodoActual = modelo.Node( nodoAAgregar, radius =  radius)
                nodoPadre.agregarHijo( nodoActual )
                for vecino in grafo.neighbors( nodoAAgregar ):
                    if vecino != numeroNodoPadre:
                        aRecorrer.append( (vecino, nodoAAgregar,nodoActual) )

 
            serial = nodoRaiz.serialize(nodoRaiz)
            #write serialized string to file
            file = open("./Trees/" + file.split(".")[0] +"_tree.dat", "w")
            file.write(serial)
            file.close() 
        else:
            print("error with file: ", file, nb, nc)


error with file:  ArteryObjAN11-5.obj 1 0
error with file:  ArteryObjAN121-11.obj 1 0
error with file:  ArteryObjAN121-8.obj 1 0
error with file:  ArteryObjAN125-13.obj 1 0
error with file:  ArteryObjAN129-16.obj 1 0
error with file:  ArteryObjAN129-5.obj 1 0
error with file:  ArteryObjAN138-16.obj 1 0
error with file:  ArteryObjAN138-19.obj 1 0
error with file:  ArteryObjAN139-0.obj 1 0
error with file:  ArteryObjAN142-7.obj 1 0
error with file:  ArteryObjAN152-12.obj 1 0
error with file:  ArteryObjAN155-15.obj 1 0
error with file:  ArteryObjAN157-3.obj 1 0
error with file:  ArteryObjAN161-9.obj 1 0
error with file:  ArteryObjAN167-5.obj 1 0
error with file:  ArteryObjAN171-17.obj 1 0
error with file:  ArteryObjAN172-8.obj 1 0
error with file:  ArteryObjAN173-16.obj 1 0
error with file:  ArteryObjAN175-9.obj 1 0
error with file:  ArteryObjAN177-6.obj 1 0
error with file:  ArteryObjAN186-6.obj 1 0
error with file:  ArteryObjAN19-0.obj 1 0
error with file:  ArteryObjAN25-15.obj 1 0
erro